In [1]:
# Choose one of the matplotlib initialization for interactive or non-interactive
%matplotlib qt5
#import matplotlib
#matplotlib.use('Qt5Agg')

In [2]:
import sqlite3
import astropy.units as u
from astropy.coordinates import SkyCoord

import os
import numpy as np
import pandas as pd
import sqlite3
import lsst.daf.persistence as dafPersist
import lsst.geom
from astropy.visualization import (ZScaleInterval, AsinhStretch, ImageNormalize)
import astropy.units as u
from astropy.table import Table
import matplotlib.pyplot as plt

In [7]:
cwpRepo = '/home/gkovacs/data/repo_DM-17825/ingested/rerun/proc_2019-02-21'
cwpTemplateRepo = '/home/gkovacs/data/repo_DM-17825/templates'
my_dbName = '/home/gkovacs/data/repo_DM-17825/ingested/rerun/proc_2019-02-21/association.db'
mrawls_dbName = '/home/gkovacs/data/repo_DM-17825/mrawls_cw_processed2/association.db'
mrawls_repo = '/home/gkovacs/data/repo_DM-17825/mrawls_cw_processed2'

butlerCwp = dafPersist.Butler(cwpRepo)
butlerCwpTemplate = dafPersist.Butler(cwpTemplateRepo)
conn = sqlite3.connect(my_dbName)

In [9]:
%run -n -i "plot_calexp_template_diffim.py"

In [4]:
patchList = ['10,8', '11,8', '12,8', '13,8',
             '10,7', '11,7', '12,7', '13,7',
             '10,9', '11,9', '12,9', '13,9',
             '10,5', '11,5', '12,5', '13,5',
             '10,6', '11,6', '12,6', '13,6',
             '10,10', '11,10', '12,10', '13,10']

In [5]:
cwpObjTable = loadAllPpdbObjects(cwpRepo)
cwpMiniRegion = defMiniRegion(cwpObjTable)
cwpMiniUnflagged = cwpMiniRegion & (cwpObjTable['flags'] == 0)

NameError: name 'loadAllPpdbObjects' is not defined

In [10]:
# Find the patch that belongs to the mini region
patch = patchFinder(cwpObjTable.loc[cwpMiniUnflagged,'diaObjectId'].values[0],cwpObjTable,butlerCwpTemplate,patchList)
print(patch)

NameError: name 'cwpObjTable' is not defined

# ==========

In [11]:
srcTable = Table.from_pandas(pd.read_sql_query('select * from  DiaSource '
                                               'where decl < -5.6 and decl > -5.8'
            ' and ra > 155.2 and ra < 155.3 and flags = 0', conn))

S = SkyCoord(srcTable['ra']*u.deg, srcTable['decl']*u.deg)

In [12]:
# Meredith's ppdb

m_conn = sqlite3.connect(mrawls_dbName)
m_srcTable = Table.from_pandas(pd.read_sql_query(
    'select * from DiaSource where decl < -5.6 and decl > -5.8'
            ' and ra > 155.2 and ra < 155.3 and flags = 0', m_conn))
m_S = SkyCoord(m_srcTable['ra']*u.deg, m_srcTable['decl']*u.deg)

In [13]:
print(pd.read_sql('select count(*) from DiaSource where decl < -5.6 and decl > -5.8'
            ' and ra > 155.2 and ra < 155.3',m_conn))

print(pd.read_sql('select count(*) from DiaSource where decl < -5.6 and decl > -5.8'
            ' and ra > 155.2 and ra < 155.3 and flags = 0',m_conn))

   count(*)
0      3486
   count(*)
0       678


In [14]:
print(pd.read_sql('select count(*) from DiaSource where decl < -5.6 and decl > -5.8'
            ' and ra > 155.2 and ra < 155.3',conn))

print(pd.read_sql('select count(*) from DiaSource where decl < -5.6 and decl > -5.8'
            ' and ra > 155.2 and ra < 155.3 and flags = 0',conn))

   count(*)
0      2697
   count(*)
0      1202


In [99]:
plot_diasources_compare(srcTable[:100],m_srcTable[:100])

In [13]:
%run -n -i "plot_diasources_comparison.py"

In [14]:
plot_diasources_diffim(cwpRepo,mrawls_repo,srcTable,m_srcTable)

In [11]:
mrawls_repo

'/home/gkovacs/data/repo_DM-17825/mrawls_cw_processed2'

In [22]:
import matplotlib
matplotlib.rcParams['lines.markersize']

6.0

In [12]:
butler2 = dafPersist.butler.Butler(mrawls_repo)

In [ ]:
idx, sep, d3d = S.match_to_catalog_sky(m_S)

In [ ]:
ccdVisitIds, visit_ctns = np.unique(srcTable['ccdVisitId'],return_counts=True)

In [ ]:
ccdVisitIds[np.argmax(visit_ctns)]

In [ ]:
T2 = srcTable[srcTable['ccdVisitId']==41980206]

In [ ]:
m_T2 = m_srcTable[m_srcTable['ccdVisitId']==41980206]

In [ ]:
fig=plt.figure()

In [ ]:
ax=fig.add_subplot(1,1,1)

In [ ]:
ax.scatter(T2['ra'],T2['decl'],s=4, alpha=0.5)

In [ ]:
ax.scatter(m_T2['ra'],m_T2['decl'],s=4, alpha=0.5)

In [17]:
diffexp = butlerCwp.get("deepDiff_differenceExp", {'visit': 419802, 'ccdnum': 6})

In [18]:
wcs = diffexp.getWcs()

In [20]:
?wcs.skyToPixel

In [81]:
p1 = lsst.geom.SpherePoint(155.3, -5.8, lsst.geom.degrees)
p2 = lsst.geom.SpherePoint(155.2, -5.6, lsst.geom.degrees)

In [82]:
p1

SpherePoint(155.3*geom.degrees, -5.8*geom.degrees)

In [30]:
img_point1 = wcs.skyToPixel(p1)
img_point2 = wcs.skyToPixel(p2)

In [83]:
img_point1, img_point2

(Point2D(1320.48885, 2368.546548), Point2D(-1420.283617, 1005.740488))

In [46]:
b_plot = lsst.geom.Box2I(lsst.geom.Point2I(img_point1),lsst.geom.Point2I(img_point2))

In [49]:
b_plot.clip(diffexp.getBBox())

In [65]:
b_plot

Box2I(minimum=Point2I(0, 1006), dimensions=Extent2I(1321, 1364))

In [52]:
C1, _, C2, _ = b_plot.getCorners()

In [66]:
C1, C2

(Point2I(0, 1006), Point2I(1320, 2369))

In [53]:
center = 0.5*(np.array(C1) + np.array(C2))

In [62]:
n_c = wcs.pixelToSky(lsst.geom.Point2D(center))

In [67]:
size = lsst.geom.Extent2I(np.fabs(np.array(C1) - np.array(C2)))

In [68]:
size

Extent2I(1320, 1363)

In [69]:
cimg = diffexp.getCutout(n_c,size)

In [70]:
cimg.getBBox()

Box2I(minimum=Point2I(0, 1006), dimensions=Extent2I(1320, 1363))

In [74]:
fig = plt.figure()

In [75]:
ax = fig.add_subplot(1,1,1)

In [78]:
ax.imshow(cimg.getMaskedImage().getImage().getArray(),extent=(0,1320,1006,1363))

In [79]:
ax.set_xlim(-1420,1320)

(-1420, 1320)

In [80]:
ax.set_ylim(1005,2369)

(1005, 2369)

In [88]:
lsst.geom.Box2I(lsst.geom.Box2D(b_plot))

Box2I(minimum=Point2I(0, 1006), dimensions=Extent2I(1321, 1364))